In [1]:
import json
import os
import polars as pl
from polars import col as c
import polars.selectors as cs
import re
from data_federation.input_model import SmallflexInputSchema
import owncloud


from data_federation.parser.market_price import parse_market_price
from data_federation.parser.discharge_flow import parse_discharge_flow
from data_federation.parser.weather import parse_weather
from data_federation.parser.aegina_hydro_power_plant import parse_aegina_hydro_power_plant
from data_federation.parser.merezenbach_hydro_power_plant import parse_merezenbach_hydro_power_plant
from data_federation.parser.morel_hydro_power_plant import parse_morel_hydro_power_plant
from utility.general_function import dictionary_key_filtering, scan_switch_directory, modify_string, generate_uuid, pl_to_dict
from utility.polars_operation import modify_string_col, generate_uuid_col

from data_federation.parser.hydro_power_plant import get_hydro_power_plant_data
from config import settings
import plotly.express as px

from itertools import product

os.chdir(os.getcwd().replace("/src", ""))




In [2]:
input_file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
market_price_metadata: dict = json.load(open(input_file_names["market_price_metadata"]))
small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()

In [3]:
kwargs: dict = {"small_flex_input_schema": small_flex_input_schema, "input_file_names": input_file_names}
# kwargs["small_flex_input_schema"] = parse_market_price(**kwargs)
# kwargs["small_flex_input_schema"] = parse_discharge_flow(**kwargs)
# kwargs["small_flex_input_schema"] = parse_weather(**kwargs, area="greis")
kwargs["small_flex_input_schema"] = parse_aegina_hydro_power_plant(**kwargs)
kwargs["small_flex_input_schema"] = parse_merezenbach_hydro_power_plant(**kwargs)
kwargs["small_flex_input_schema"] = parse_morel_hydro_power_plant(**kwargs)
# kwargs["small_flex_input_schema"] 

In [5]:
kwargs["small_flex_input_schema"]

SmallflexInputSchema(humidity=shape: (0, 4)
┌───────────────────┬───────────┬──────────────┬───────┐
│ timestamp         ┆ sub_basin ┆ start_height ┆ value │
│ ---               ┆ ---       ┆ ---          ┆ ---   │
│ datetime[μs, UTC] ┆ str       ┆ i32          ┆ f64   │
╞═══════════════════╪═══════════╪══════════════╪═══════╡
└───────────────────┴───────────┴──────────────┴───────┘, basin_height=shape: (1_826, 3)
┌─────────────────────────────────┬─────────────────────────┬─────────┐
│ water_basin_fk                  ┆ timestamp               ┆ height  │
│ ---                             ┆ ---                     ┆ ---     │
│ str                             ┆ datetime[μs, UTC]       ┆ f64     │
╞═════════════════════════════════╪═════════════════════════╪═════════╡
│ f7ef6d1c-fa81-556e-8c39-66cf9a… ┆ 2018-10-01 00:00:00 UTC ┆ 2384.0  │
│ f7ef6d1c-fa81-556e-8c39-66cf9a… ┆ 2018-10-02 00:00:00 UTC ┆ 2384.0  │
│ f7ef6d1c-fa81-556e-8c39-66cf9a… ┆ 2018-10-03 00:00:00 UTC ┆ 2384.0  │
│ f7ef

In [4]:
file_name = ".cache/input/greis_fmv_input_data/KWA_Seeniveau_2018-2023.csv"

data: pl.DataFrame = pl.read_csv(file_name, separator=";", infer_schema_length=0).select(
    c("Date").str.to_datetime(format="%d.%m.%Y", time_zone="UTC").alias("timestamp"),
    c("Last").str.replace(",", ".").cast(pl.Float64).alias("volume"),
    pl.li()
)



AttributeError: module 'polars' has no attribute 'li'

In [5]:
for name in  ["swissgrid_cap"]:
    for entry in list(os.scandir(market_price_metadata[name]["folder"])):

        data: pl.DataFrame = pl.read_csv(entry.path, separator=";", encoding='iso-8859-1', null_values=["*", "N/A"])
        # We remove weekly-based market prices
        data.rename(market_price_metadata[name]["col_mapping"])\
            .filter(pl.col("quantity") > 0)\
            .filter(~ pl.col("metadata").str.contains("KW"))
        data = data.with_columns(
            pl.col("time").str.split(" bis ").list.get(0).str.split(" ").list.get(-1).alias("time"),
            pl.col("metadata").str.split_exact("_", 4).struct.rename_fields(["market", "y", "m", "d"]),
        ).unnest("metadata")\
        .with_columns(
            pl.concat_str(["y", "m", "d", "time"], separator="-")
            .str.to_datetime("%y-%m-%d-%H:%M", time_zone="UTC").alias("timestamp"),
        ).group_by(["timestamp", "market"]).agg(
            c("price").min().alias("min"),
            ((c("price") *c("quantity")).sum()/c("quantity").sum()).alias("avg"),
            c("price").max().alias("max"),
        ).with_columns(
            c("market").replace_strict(market_price_metadata[name]["market_mapping"])
        ).with_columns(
            pl.lit(value).alias(name) for name, value in market_price_metadata[name]["data"].items()
        ).unnest("market")

# data = data.rename(market_price_metadata[name]["col_mapping"]).filter(pl.col("quantity") > 0)

ColumnNotFoundError: time

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["Ausschreibung", "Beschreibung", "Angebotenes Volumen", "Einheit"]; PROJECT */16 COLUMNS; SELECTION: None

In [ ]:
head_table

In [ ]:
data_pl

In [ ]:
data_pl.select(
    pl.struct(cs.contains("Pe_") != 0).alias("resource_state")
)

In [ ]:
precipitation = data.unpivot(
        index=["timestamp"], variable_name="metadata", value_name="value", on=cs.starts_with("prec_")
    ).with_columns(
    ("gries_" + c("metadata").str.slice(-3, 1)).alias("sub_basin"),
    ((c("metadata").str.slice(-2) + "00").cast(pl.Int32)-50).alias("start_height")
)


In [ ]:
precipitation

In [ ]:
data.columns

In [ ]:
small_flex_input_schema

In [ ]:
discharge_flow

In [ ]:
data_str = str(ro.r['df'])
data_str = (list(map(lambda x: modify_string(x, format_str={r"\s+": " "}).split(" "), data_str.split("\n"))))


index = [i for i in range(len(data_str)) if data_str[i][0] == ""]
for i in range(len(index)-2):
    if i == 0:
        columns = ["index", "date"] + data_str[index[i]][1:]
        data: pl.DataFrame = pl.DataFrame(data_str[index[i] + 1:index[i+1]], schema=columns, orient="row")
    else:
        columns = ["index"] + data_str[index[i]][1:]
        data = data.join(
            pl.DataFrame(data_str[index[i] + 1:index[i+1]], schema=columns, orient="row"), on="index", how="left")


In [ ]:

data = data.with_columns(
    pl.concat_str(["date", "time"], separator=" ").str.to_datetime("%Y-%m-%d %H:%M:%S", time_zone="UTC").alias("timestamp")
)

wind_data = data.unpivot(on=cs.contains("wind"), index="timestamp", value_name="wind_speed", variable_name="height")\
    .with_columns(
        c("height").str.replace("wind_X", ""),
        c("wind_speed").cast(pl.Float64),
    )

irradiance_data = data.unpivot(on=cs.contains("glob"), index="timestamp", value_name="irradiance", variable_name="height")\
    .with_columns(
        c("height").str.replace("glob__X", ""),
        c("irradiance").cast(pl.Float64),
    )

irradiance_data

In [ ]:
data

In [ ]:
list(map(lambda x: x.split(" "), data_str.split("\n")))


In [ ]:
data = pl.from_pandas(pandas2ri.rpy2py())
print(data.head())

In [ ]:
ro.r['df']

In [ ]:
df_temp2 =data.filter(pl.col("metadata").str.contains("KW"))\
    .with_columns(
        c("metadata").str.replace("KW", "").
        str.split_exact("_", 4).struct.rename_fields(["market", "y", "w"]).alias("metadata"),
    ).unnest("metadata").with_columns(
        (c("y").str.to_datetime("%y", time_zone="UTC") + 
        ((pl.col("w").cast(pl.Int32) - 1)*7*24*60*60*1e3).cast(pl.Duration(time_unit="ms"))).alias("timestamp"),
    )
df_temp2.head(1)


In [ ]:
    .with_columns(
        pl.col("Date").str.split("_").map_elements(lambda x: x[0]).alias("market"),
        pl.col("Date").str.split("_").map_elements(lambda x: x[1]).alias("y").cast(pl.Int64),
        pl.col("Date").str.split("KW").map_elements(lambda x: x[-1]).alias("week").str.split("_")
        .map_elements(lambda x: x[0]).cast(pl.Int64)]).with_columns(
            ((pl.col("week") - 1)*7*24*60*60*1e6).cast(pl.Duration(time_unit="us")),
                (pl.col("y") + 2000).cast(pl.Utf8).str.strptime(pl.Datetime, "%Y")
            )


In [ ]:
name = "rte_ene"
market_price: pl.DataFrame = pl.read_csv(
    market_price_metadata[name]["file"], separator=";", null_values=["*"])

market_price.columns